The following code segment creates key files for all applicable slots and programs it into the device.

 # TrustFLEX device resource generator/programmer

### Prerequisites:
The following code is runs on python 3.x environment. The folowing modules are also needed to run the example

1. cryptoauthlib
2. cryptography

The next step of the tutorial with attempt to load all the necessary modules and their dependencies on your machine. If the above modules are already installed you can safely step over the next Tutorial step.

> <font color = green>pip install -U module_name</font>

In [ ]:
!python -m pip install -r ../requirements.txt

### Load the necessary modules and helper functions

This Tutorial uses a number of python helper functions that are loaded by the following cell.
The below code does the following things:
1. Setup of cryptography module
2. Adds helper functions to get device name and device type

In [ ]:
from common import *
from helper import *
from manifest_signing_helper import *
import os, fnmatch
import cryptography
from cryptoauthlib import *
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')
print("Importing modules - Successfull")

### Establish the communication between host (PC) and TrustFLEX
1. Initialize the library which also loads it
2. Check if the correct device is connected to the PC. If the wrong device is connected, a ValueError excpetion will be raised.

In [ ]:
ATCA_KIT_I2C_IFACE = 1
cfg = cfg_ateccx08a_kithid_default()
cfg.devtype = get_device_type_id('ATECC608A')
cfg.cfg.atcahid.dev_interface = ATCA_KIT_I2C_IFACE
cfg.cfg.atcahid.dev_identity = 0x6C #TFLXTLS device I2C address
ATCA_SUCCESS = 0

Establish the connection with the ECC608A-TLXTLS device on the USB dongle and check its type
<font color=red>If the next Tutorial step yields an exception, the ECC608A is not correctly connected or of the wrong type.</font>

In [ ]:
# Initialize interface
assert atcab_init(cfg) == ATCA_SUCCESS, "Can't connect to the USB dongle"

# Get connected device type
info = bytearray(4)
assert atcab_info(info) == ATCA_SUCCESS, "Can't read the ECC608A device information"
dev_type = get_device_type_id(get_device_name(info))


# Checking if the connected device matches with selected device
if dev_type != cfg.devtype:
    assert atcab_release() == ATCA_SUCCESS
    raise ValueError('Device not supported')
    
print("Device initialization - Successfull")

The following code segment creates key files for all applicable slots and programs it into the device. 

<font color=green>The keys are generated here using python module and is stored in computer's hard disk, this is not secure and is not recommended for actual production. In actual production environment these keys need to be stored in a secure storage as depitected below.</font>

In [ ]:
print('\n\r---------------------------------------------\n\r')
slots = [0,1,2,3,4,6,5,7,8,9,10,11,12,13,14,15] 
for i in slots:
    status = create_resource_and_program(i)
    if(status != True):
        raise ValueError("Resource gererator failure")

# Additional step to generate files for C-examples (ignore for Notebooks)
create_resources_for_c_examples();

print('\n\r---------------------------------------------\n\r')
print('Key generation - Success')
print('\n\r---------------------------------------------\n\r')

The following code creates a signing key and certificate that will be used to sign entries in a manifest. Manifests downloaded from Microchip Direct will be signed by Microchip

In [ ]:
create_log_signer()

The following code segment manages to insert customer parameters for common name and organization name into certificate templates

In [ ]:
#Get cert type
print ('Choose certificate type\n\r')
print ('\t1 - MCHP standard certificate\n\r')
print ('\t2 - Custom certificate\n\r')
cert_type = input('Enter certificate type\n\r')

if (cert_type == '1'):
    print ('\n\r---------------------------------------------\n\r')
    print ('MCHP certificate related files are pre-generated')
    print ('\n\r---------------------------------------------\n\r')
    mchp_cert()
elif (cert_type == '2'):
    custom_cert()
else:
    raise ValueError("Invalid cert_type entry")
    
print('\n\r---------------------------------------------\n\r')
print('Certificate related steps completed successfully')
print('\n\r---------------------------------------------\n\r')

After executing the above code successfully you will find resources generated for all slots. 

Keys are generated and stored in the device, the same key will be available in the the same directory as .pem files. For certificates the tool will generate keys, certificates and associated .c and .h files. 
    
<font color=green> This text marks the end of this tool, after this you can run the usecase examples.